<a href="https://colab.research.google.com/github/cognitedata/WiDS-2019/blob/master/WiDS_2019_Cognite_Interact_with_Assets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Classification Methods to Label Industrial Data

## What this notebook will achieve

* Extract data from an oil rig in the North Sea.
* Add more later




## Getting started

* Having a basic understanding of Python concepts will help to understand the process.

* Cognite has released *live* data to the public on the Cognite Data Platform streaming from [Valhall](https://www.akerbp.com/en/our-assets/production/valhall/), one of Aker's oil fields.

* To access the data, generate an API Key on [Open Industrial Data](https://openindustrialdata.com/). Get your key via the Google Access platform. You will be asked to fill out some personal information to generate your personal key.

* Visualize some of the machines (assets) on Valhall with Cognite's [Operational Intelligence](https://opint.cogniteapp.com/publicdata/infographics/-LOHKEJPLvt0eRIZu8mE) dashboard. This data on this page shows is streaming live data from the Valhall oil field located in the North Sea.

* To understand how to interact with the data using the Python SDK ([Docs](https://cognite-sdk-python.readthedocs-hosted.com/en/latest/)) follow along in this notebook.

## Environment Setup

#### Install the Cognite SDK package

In [ ]:
!pip install cognite-sdk

#### Import the required packages

In [ ]:
import os
from getpass import getpass

import numpy as np
import pandas as pd

from cognite.client import CogniteClient

#### Connect to the Cognite Data Platform
* This client object is how all queries will be sent to the Cognite API to retrieve data.

When prompted for your API key, use the key generated by open industrial data as mentioned in the Getting Started steps.

In [ ]:
client = CogniteClient(api_key=getpass("Open Industrial Data API-KEY: "),
                       project="publicdata", client_name="OID_example")

## Accessing Cognite Data Platform (CDP)
* The CDP organizes digital information about the physical world.
* There are 6 kinds of objects stored on the CDP. Each of these objects in the CDP are labelled with a unique ID. Information regarding a specific Asset, Event, etc are often retrieved using this ID.

  * [Assets](https://doc.cognitedata.com/api/0.5/#tag/Assets) are digital representations of physical objects or groups of objects, and assets are organized into an asset hierarchy. For example, an asset can represent a water pump which is part of a subsystem on an oil platform.
  
  * [Event](https://doc.cognitedata.com/api/0.5/#tag/Events) objects store complex information about multiple assets over a time period. For example, an event can describe two hours of maintenance on a water pump and some associated pipes.
  
  * A [File](https://doc.cognitedata.com/api/0.5/#tag/Files) stores a sequence of bytes connected to one or more assets. For example, a file can contain a piping and instrumentation diagram (P&IDs) showing how multiple assets are connected.
  
  * A [Time Series](https://doc.cognitedata.com/api/0.5/#tag/Time-series) consists of a sequence of data points connected to a single asset. For example: A water pump asset can have a temperature time series that records a data point in units of °C every second.
  
  * [Sequence](https://doc.cognitedata.com/api/0.5/#tag/Sequences) are similar to time series in that they are a key value pair, but rather than using a timestamp as the key, another measurment such as depth could be the key. For example, this is used in practice when drilling and taking measurments at various depths.
  
  * A [3D](https://doc.cognitedata.com/api/0.5/#tag/3D) model is typically built up by a hierarchical structure. This looks very similar to how we organize our internal asset hierarchy. 3D models are visualized via Cognite's dashboards.
  
* It is important to refer back to the [SDK](https://cognite-sdk-python.readthedocs-hosted.com/en/latest/cognite.html) for specific details on arguments on all avaiable methods on how to access these objects.

### Collecting Asset Information

#### Retrieve a list of all Assets

* There are thousands of Assets in the CDP, we can have a look at a few examples.

* This will generate a list of assets from the CDP with no particular filters, this is a random result. Generally we would want to apply filters when retrieving records.


In [ ]:
client.assets.list().to_pandas().head()

#### Decide on which asset we want to explore
* To get started exploring data in the CDP, we must first decide on which Asset we want to gather information from.

* Some asset names may be retrieved from the [Op Int](https://opint.cogniteapp.com/publicdata/infographics/-LOHKEJPLvt0eRIZu8mE) dashboard.

* Here is a screehshot of the [OpInt Dashboard](https://drive.google.com/open?id=1f_7nJaJu5Xgr3Oq09mIZ0KwjBAYbzEUQ) incase the page does not load.

* Some example asset names are:
  * 23-HA-9103
  * 23-PV-92583
  * 23-VG-9101
  
The *fuzzy* search for an asset can be performed as followed


In [ ]:
asset_name = "23-HA-9103"
asset_df = client.assets.search(name=asset_name).to_pandas()
asset_df.head()


#### Get information on the asset of interest

* We can filter the assets to get asset-specific details based on asset_name

* The *get_asset()* interface provides the same information for 1 specific asset based on the provided ID


In [ ]:
asset_id = asset_df[asset_df["name"] == asset_name].iloc[0]['id']
asset = client.assets.retrieve(id=asset_id).to_pandas()
asset

#### How do we get Asset relationships?

* The interface *get_asset_subtree()* can be used to retrieve the *children* of an Asset. 

* Each Asset is given various properties, some of the useful ones for this method are:

  * Depth: The number of edges from the parent node
  
  * Description: Includes information such as the platform and type of sensor being monitored
  
We will generate a list of all children of the main asset of interest. This is done by specifying a depth of 1.

In [ ]:
subtree_df = client.assets.retrieve_subtree(id=asset_id, depth=1).to_pandas()
subtree_df.head()